In [ ]:
import os
import pandas as pd
from google.colab import drive
import re

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Texts/Texts/'


header = ['Author', 'Hotel_ID', 'Season', 'Value', 'Rooms', 'Location', 'Cleanliness', 'Checkin/Front Desk', 'Service', 'Business Service', 'Overall']


data = []
month_to_season = {
    'Jan': 'Winter', 'Feb': 'Winter', 'Mar': 'Spring',
    'Apr': 'Spring', 'May': 'Spring', 'Jun': 'Summer',
    'Jul': 'Summer', 'Aug': 'Summer', 'Sep': 'Autumn',
    'Oct': 'Autumn', 'Nov': 'Autumn', 'Dec': 'Winter'
}


for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        hotel_id = filename.split('_')[1].split('.')[0]

        text_file_path = os.path.join(folder_path, filename)

        with open(text_file_path, 'r') as file:
            lines = file.readlines()

        author = ''
        ratings = []
        month = ''

        for line in lines:
            line = line.strip()

            if line.startswith("<Author>"):
                if author:
                    review_data = {'Author': author, 'Hotel_ID': hotel_id}
                    review_data['Season'] = month_to_season.get(month[:3], 'Unknown')
                    for aspect, rating in zip(header[3:], ratings):
                        review_data[aspect] = rating
                    data.append(review_data)

                author = line[8:]
                ratings = []
                month = ''
            elif line.startswith("<Date>"):
                match = re.search(r'<Date>([A-Za-z]+)', line)
                if match:
                    month = match.group(1)

            elif line.startswith("<Rating>"):
                ratings = line[8:].split('\t')

        if author:
            review_data = {'Author': author, 'Hotel_ID': hotel_id}
            review_data['Season'] = month_to_season.get(month[:3], 'Unknown')
            for aspect, rating in zip(header[3:], ratings):
                review_data[aspect] = rating
            data.append(review_data)


df = pd.DataFrame(data, columns=header)


In [ ]:
df

,Author,Hotel_ID,Season,Value,Rooms,Location,Cleanliness,Checkin/Front Desk,Service,Business Service,Overall
0,travellingmum0,257287,Winter,5,5,5,5,5,5,5,5
1,grantosphotos,257287,Winter,4,5,4,5,5,4,4,-1
2,jmac65,257287,Autumn,5,5,5,5,5,5,5,4
3,corrigin,257287,Summer,5,5,4,5,4,5,5,4
4,albie-joh,257287,Summer,5,5,4,5,5,5,4,4
...,...,...,...,...,...,...,...,...,...,...,...
246393,CTSammyB,256734,Autumn,5,4,5,-1,5,-1,5,-1
246394,dritter,256734,Autumn,5,-1,-1,-1,-1,-1,-1,-1
246395,jurypro,256734,Autumn,5,-1,-1,-1,-1,-1,-1,-1
246396,A TripAdvisor Member,256734,Summer,5,-1,-1,-1,-1,-1,-1,-1


In [ ]:
df['Hotel_ID'].nunique()

1850

In [ ]:
import pandas as pd
from scipy import stats

data = df

data['Value'] = pd.to_numeric(data['Value'], errors='coerce')

def split_item(item_df, context):
    target_context_data = item_df[item_df['Season'] == context]
    other_contexts_data = item_df[item_df['Season'] != context]

    if len(target_context_data) > 1:
        # t-test
        _, p_value = stats.ttest_ind(target_context_data['Value'], other_contexts_data['Value'])
        if p_value < 0.05:
            return target_context_data, other_contexts_data
    return None, None

contextual_dataframes = {}
for context in data['Season'].unique():
  contextual_dataframes[context] = {'target_context': [], 'other_contexts': []}
  for item_id, item_df in data.groupby('Hotel_ID'):
    target_context_subset, other_contexts_subset = split_item(item_df, context)
    if target_context_subset is not None and other_contexts_subset is not None:
      contextual_dataframes[context]['target_context'].append(target_context_subset)
      contextual_dataframes[context]['other_contexts'].append(other_contexts_subset)


context_data = {}
for context, splits in contextual_dataframes.items():
    target_context_data = pd.concat(splits['target_context'])
    other_contexts_data = pd.concat(splits['other_contexts'])
    context_data[context] = {'target_context': target_context_data, 'other_contexts': other_contexts_data}


/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment 

In [ ]:
target_context_data

,Author,Hotel_ID,Season,Value,Rooms,Location,Cleanliness,Checkin/Front Desk,Service,Business Service,Overall
129228,coalbank,100509,Spring,2,1,2,3,1,2,2,3
129229,orly_8,100509,Spring,2,-1,-1,-1,-1,-1,-1,-1
129230,sada74,100509,Spring,4,5,3,5,4,5,4,-1
129231,DCDavidG,100509,Spring,3,2,2,4,2,4,3,3
129232,triclops,100509,Spring,4,5,4,3,5,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...
127659,DaSaint,98641,Spring,5,5,5,5,5,4,4,5
127663,A TripAdvisor Member,98641,Spring,5,5,5,-1,5,-1,5,-1
127664,Welsh-Dragon-2006,98641,Spring,4,4,4,-1,4,-1,4,-1
127671,HongKongMember,98641,Spring,3,-1,-1,-1,-1,-1,-1,-1


In [ ]:

target_context_data = target_context_data.rename(columns={'Checkin/Front Desk': 'Checkin'})
target_context_data = target_context_data.rename(columns={'Business Service': 'Business_Service'})


In [ ]:
target_context_data

,Author,Hotel_ID,Season,Value,Rooms,Location,Cleanliness,Checkin,Service,Business_Service,Overall
129228,coalbank,100509,Spring,2,1,2,3,1,2,2,3
129229,orly_8,100509,Spring,2,-1,-1,-1,-1,-1,-1,-1
129230,sada74,100509,Spring,4,5,3,5,4,5,4,-1
129231,DCDavidG,100509,Spring,3,2,2,4,2,4,3,3
129232,triclops,100509,Spring,4,5,4,3,5,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...
127659,DaSaint,98641,Spring,5,5,5,5,5,4,4,5
127663,A TripAdvisor Member,98641,Spring,5,5,5,-1,5,-1,5,-1
127664,Welsh-Dragon-2006,98641,Spring,4,4,4,-1,4,-1,4,-1
127671,HongKongMember,98641,Spring,3,-1,-1,-1,-1,-1,-1,-1


In [ ]:
other_contexts_data

,Author,Hotel_ID,Season,Value,Rooms,Location,Cleanliness,Checkin/Front Desk,Service,Business Service,Overall
129211,New_Traveler_1970,100509,Winter,5,4,5,4,5,5,5,3
129212,nicolemxoxo,100509,Winter,4,4,3,5,4,5,5,-1
129213,gadget1,100509,Winter,4,-1,-1,-1,-1,-1,-1,-1
129214,tannermae,100509,Autumn,2,5,3,5,1,2,2,-1
129215,GriffoBarcelona,100509,Autumn,3,3,2,5,3,4,3,-1
...,...,...,...,...,...,...,...,...,...,...,...
127674,A TripAdvisor Member,98641,Summer,1,-1,-1,-1,-1,-1,-1,-1
127675,A TripAdvisor Member,98641,Summer,1,-1,-1,-1,-1,-1,-1,-1
127676,A TripAdvisor Member,98641,Autumn,3,-1,-1,-1,-1,-1,-1,-1
127677,lass=,98641,Summer,5,-1,-1,-1,-1,-1,-1,-1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Flatten, Dense, Concatenate, Input


# Assuming 'target_context_data' is your dataframe
df = target_context_data

criteria_columns = ['Checkin', 'Value', 'Rooms', 'Cleanliness', 'Location', 'Business_Service', 'Service']

models = {}
predictions = {}

# Replace -1 with NaN and convert to numeric for all criteria columns
for criteria in criteria_columns:
    df[criteria].replace(-1, np.nan, inplace=True)
    df[criteria] = pd.to_numeric(df[criteria], errors='coerce')
    df[criteria].fillna(-1, inplace=True)  # Set NaN to integer -1

# Encode 'Author' and 'Hotel_ID'
author_encoder = LabelEncoder()
hotel_encoder = LabelEncoder()
df['Author_encoded'] = author_encoder.fit_transform(df['Author'])
df['Hotel_ID_encoded'] = hotel_encoder.fit_transform(df['Hotel_ID'])

embedding_dim = 32

user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(len(author_encoder.classes_), embedding_dim)(user_input)
item_embedding = Embedding(len(hotel_encoder.classes_), embedding_dim)(item_input)

user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)

x = Concatenate()([user_flatten, item_flatten])

# Define DNN model
dense_layer1 = Dense(128, activation='relu')(x)
dense_layer2 = Dense(128, activation='relu')(dense_layer1)
output_layer = Dense(1, activation='relu')(dense_layer2)

# Model
model = Model(inputs=[user_input, item_input], outputs=output_layer)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Loop through each criteria to fit the model
for criteria in criteria_columns:
    X_user = df['Author_encoded'].values.reshape(-1, 1)
    X_item = df['Hotel_ID_encoded'].values.reshape(-1, 1)
    y = df[criteria].values

    # Split data into training and validation sets randomly
    X_user_train, X_user_val, X_item_train, X_item_val, y_train, y_val = train_test_split(
        X_user, X_item, y, test_size=0.2, random_state=42
    )

    history = model.fit(
        [X_user_train, X_item_train], y_train,
        epochs=50, batch_size=32,
        validation_data=([X_user_val, X_item_val], y_val)
    )

    predictions[criteria] = model.predict([X_user, X_item])

# Predictions of ratings for all the criteria for a particular context
rpred_checkin = predictions['Checkin']
rpred_value = predictions['Value']
rpred_rooms = predictions['Rooms']
rpred_cleanliness = predictions['Cleanliness']
rpred_location = predictions['Location']
rpred_business_service = predictions['Business_Service']
rpred_service = predictions['Service']


Epoch 1/49
164/164 [==============================] - 3s 7ms/step - loss: 6.7810 - mae: 2.2503 - val_loss: 5.2898 - val_mae: 2.0102
Epoch 2/49
164/164 [==============================] - 1s 5ms/step - loss: 4.4366 - mae: 1.7175 - val_loss: 5.5562 - val_mae: 2.0953
Epoch 3/49
164/164 [==============================] - 1s 5ms/step - loss: 2.8471 - mae: 1.3291 - val_loss: 6.1600 - val_mae: 2.2528
Epoch 4/49
164/164 [==============================] - 1s 5ms/step - loss: 2.4640 - mae: 1.2063 - val_loss: 5.5567 - val_mae: 2.0991
Epoch 5/49
164/164 [==============================] - 1s 5ms/step - loss: 2.2418 - mae: 1.0767 - val_loss: 5.4907 - val_mae: 2.0546
Epoch 6/49
164/164 [==============================] - 1s 5ms/step - loss: 2.1445 - mae: 0.9933 - val_loss: 5.5261 - val_mae: 2.0993
Epoch 7/49
164/164 [==============================] - 1s 5ms/step - loss: 2.1276 - mae: 0.9857 - val_loss: 5.5741 - val_mae: 2.1241
Epoch 8/49
164/164 [==============================] - 1s 5ms/step - loss: 2.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Concatenate, Input, InputLayer
from tensorflow.keras.optimizers import Adam

# Assuming 'target_context_data' is your dataframe
df = target_context_data

criteria_columns = ['Checkin', 'Value', 'Rooms', 'Cleanliness', 'Location', 'Business_Service', 'Service']
predictions = {}
criteria_stats = {}

# Replace -1 with NaN and convert to numeric for all criteria columns
for criteria in criteria_columns:
    df[criteria].replace(-1, np.nan, inplace=True)
    df[criteria] = pd.to_numeric(df[criteria], errors='coerce')
    df[criteria].fillna(-1, inplace=True)  # Set NaN to integer -1

# Encode 'Author' and 'Hotel_ID'
author_encoder = LabelEncoder()
hotel_encoder = LabelEncoder()
df['Author_encoded'] = author_encoder.fit_transform(df['Author'])
df['Hotel_ID_encoded'] = hotel_encoder.fit_transform(df['Hotel_ID'])

embedding_dim = 32

user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(len(author_encoder.classes_), embedding_dim)(user_input)
item_embedding = Embedding(len(hotel_encoder.classes_), embedding_dim)(item_input)

user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)

x = Concatenate()([user_flatten, item_flatten])

# Define DNN model
dense_layer1 = Dense(128, activation='relu')(x)
dense_layer2 = Dense(128, activation='relu')(dense_layer1)
output_layer = Dense(1, activation='relu')(dense_layer2)

# Model
model = Model(inputs=[user_input, item_input], outputs=output_layer)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Loop through each criteria to fit the model
for criteria in criteria_columns:
    X_user = df['Author_encoded'].values.reshape(-1, 1)
    X_item = df['Hotel_ID_encoded'].values.reshape(-1, 1)
    y = df[criteria].values

    # Split data into training and validation sets randomly
    X_user_train, X_user_val, X_item_train, X_item_val, y_train, y_val = train_test_split(
        X_user, X_item, y, test_size=0.2, random_state=42
    )

    history = model.fit(
        [X_user_train, X_item_train], y_train,
        epochs=50, batch_size=32,
        validation_data=([X_user_val, X_item_val], y_val)
    )

    predictions[criteria] = model.predict([X_user, X_item])

# Calculate statistics for normalization
for criteria in criteria_columns:
    mean = np.mean(predictions[criteria])
    std = np.std(predictions[criteria])
    criteria_stats[criteria] = {'mean': mean, 'std': std}

# Normalize predictions
rpred_normalized = {}
for criteria in criteria_columns:
    mean = criteria_stats[criteria]['mean']
    std = criteria_stats[criteria]['std']
    rpred_normalized[criteria] = (predictions[criteria] - mean) / std

# Stack normalized predictions to create input for the second model
x = np.column_stack([rpred_normalized[criteria] for criteria in criteria_columns])

# Prepare the target variable
df['Overall'].replace(-1, np.nan, inplace=True)
df['Overall'] = pd.to_numeric(df['Overall'], errors='coerce')
df['Overall'].fillna(-1, inplace=True)
y = df['Overall'].values

# Split data into training and validation sets randomly
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# DNN model for overall ratings prediction
model_overall = Sequential([
    InputLayer(input_shape=(len(criteria_columns),)),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

model_overall.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

history = model_overall.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_val, y_val))

# Make predictions
predictions_overall = model_overall.predict(x)


Epoch 1/150
164/164 [==============================] - 1s 4ms/step - loss: 2.5571 - mae: 1.0239 - val_loss: 4.3471 - val_mae: 1.5858
Epoch 2/150
164/164 [==============================] - 0s 3ms/step - loss: 2.4389 - mae: 0.9683 - val_loss: 4.3443 - val_mae: 1.5692
Epoch 3/150
164/164 [==============================] - 0s 3ms/step - loss: 2.4072 - mae: 0.9546 - val_loss: 4.2713 - val_mae: 1.5791
Epoch 4/150
164/164 [==============================] - 0s 3ms/step - loss: 2.4031 - mae: 0.9516 - val_loss: 4.2383 - val_mae: 1.5859
Epoch 5/150
164/164 [==============================] - 0s 3ms/step - loss: 2.3806 - mae: 0.9496 - val_loss: 4.2091 - val_mae: 1.5740
Epoch 6/150
164/164 [==============================] - 0s 3ms/step - loss: 2.3730 - mae: 0.9457 - val_loss: 4.1969 - val_mae: 1.5944
Epoch 7/150
164/164 [==============================] - 0s 3ms/step - loss: 2.3638 - mae: 0.9406 - val_loss: 4.1886 - val_mae: 1.5830
Epoch 8/150
164/164 [==============================] - 0s 3ms/step - 

In [ ]:

target_context_data.reset_index(drop=True, inplace=True)


In [ ]:
predictions_df = pd.DataFrame(predictions_overall, columns=['Predicted_Overall'])

#Adding columns in the start
predictions_df['Author'] = target_context_data['Author']
predictions_df['Hotel_ID'] = target_context_data['Hotel_ID']

In [ ]:
predictions_df

,Predicted_Overall,Author,Hotel_ID
0,2.432319,coalbank,100509
1,-0.975153,orly_8,100509
2,0.750127,sada74,100509
3,2.606215,DCDavidG,100509
4,4.305480,triclops,100509
...,...,...,...
6553,4.047625,DaSaint,98641
6554,-1.007136,A TripAdvisor Member,98641
6555,2.906466,Welsh-Dragon-2006,98641
6556,3.589954,HongKongMember,98641


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# True labels
true_labels = target_context_data['Overall']

# Mean Absolute Error
mae = mean_absolute_error(true_labels, predictions_overall)

# Root Mean Squared Error
rmse = mean_squared_error(true_labels, predictions_overall, squared=False)

# Print the results
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("The accuracy of our model is {}%".format(round(r_squared, 2) * 100))


In [ ]:
# Step 1: Filter the DataFrame based on the desired Author
author_id = 'A TripAdvisor Member'
author_df = predictions_df[predictions_df['Author'] == author_id]

# Step 2: Extract unique items rated by the author
unique_items_rated = author_df['Hotel_ID'].unique()

print("Unique items rated by {}: {}".format(author_id, unique_items_rated))

# Step 3: Find the item with the maximum predicted rating
max_rating_item = author_df.loc[author_df['Predicted_Overall'].idxmax()]

print("\nItem with the maximum predicted rating by {}: \n{}".format(author_id, max_rating_item))
